d ##### Usecase:Get sales data by orders
###### Solutions are given for following problem statements
###### Problem statement-1:Get sales data per day
###### Problem statement-2:Get sales data per month
###### Problem statement-3:Get sales date per year
###### Problem statement-4:Get monthly sales data from third quarter of 2013 and first quarter of 2014 when Max sales >100 
###### Problem statement-5:Get monthly sales data from last quarter of 2013 where soldquantity_permonth > 25000 
###### Problem statement-6:Get daily sales data from fourth quarter of 2013 and second quarter of 2014 where averagesale > 200 and Numberofsales >=600
###### Problem statement-7:Get daily sales data  where soldquantity_perday > 1500
###### Problem statement-8:Get final qurater of 2013 sales data of TOP5 revenue generating order for each month
###### Problem statement-9:Get average revenue per day and all the orders which are more than average.
###### Problem statement-10:Percentage of order items in Order revenue

In [2]:
from pyspark.sql import SQLContext, Row
from pyspark.sql import HiveContext
from pyspark.sql.functions import round,sum,avg,count,substring
hiveContext = HiveContext(sc)
sqlContext.sql("set spark.sql.shuffle.partitions=10")

Out[2]: DataFrame[key: string, value: string]

In [3]:
hiveContext.sql("create database retail_db")
hiveContext.sql("use retail_db")

In [4]:
hiveContext.sql("create table orders(order_id int,order_date string,order_customer_id int, order_status string) row format delimited fields terminated by ','")
hiveContext.sql("create table order_items(order_item_id int, order_item_order_id int, order_item_product_id  int, order_item_quantity int,order_item_subtotal float,order_item_product_price float) row format delimited fields terminated by ','")

In [5]:
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Orders.txt' into table orders")
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Order_Items.txt' into table order_items")

In [6]:
hiveContext.sql("show tables").show()

+---------+-----------+-----------+
 database| tableName|isTemporary|
+---------+-----------+-----------+
retail_db| categories| false|
retail_db| customers| false|
retail_db|departments| false|
retail_db|order_items| false|
retail_db| orders| false|
retail_db| products| false|
+---------+-----------+-----------+

In [7]:
hiveContext.sql("select * from orders").show(5,False)

+--------+-----------------------+-----------------+---------------+
order_id|order_date |order_customer_id|order_status |
+--------+-----------------------+-----------------+---------------+
1 |2013-07-25 00:00:00.000|11599 |CLOSED |
2 |2013-07-25 00:00:00.000|256 |PENDING_PAYMENT|
3 |2013-07-25 00:00:00.000|12111 |COMPLETE |
4 |2013-07-25 00:00:00.000|8827 |CLOSED |
5 |2013-07-25 00:00:00.000|11318 |COMPLETE |
+--------+-----------------------+-----------------+---------------+
only showing top 5 rows

In [8]:
hiveContext.sql("select * from order_items").show(5,False)


+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

###### Problem statement-1:Get sales data per day

In [10]:
SalesDataPerDay = hiveContext.sql('''select cast(otl.order_date as DATE) as orderdate,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales, 
                                   max(oitl.order_item_subtotal) as Max_sales, 
                                   count(oitl.order_item_subtotal) as Numberofsales_perday, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_perday 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                   group by otl.order_date 
                                   order by otl.order_date''')
SalesDataPerDay.coalesce(1).write.csv("/FileStore/tables/SalesDataPerDay", compression="none", header ='true')
SalesDataPerDay.show(20,False)

+----------+------------+---------+---------+--------------------+---------------+
orderdate |Totalrevenue|Min_sales|Max_sales|Numberofsales_perday|avgsales_perday|
+----------+------------+---------+---------+--------------------+---------------+
2013-07-25|68153.83 |100.0 |99.99 |339 |201.04 |
2013-07-26|136520.17 |100.0 |99.99 |694 |196.71 |
2013-07-27|101074.34 |100.0 |99.99 |503 |200.94 |
2013-07-28|87123.08 |100.0 |99.99 |438 |198.91 |
2013-07-29|137287.09 |100.0 |99.99 |666 |206.14 |
2013-07-30|102745.62 |100.0 |99.99 |540 |190.27 |
2013-07-31|131878.06 |100.0 |99.99 |641 |205.74 |
2013-08-01|129001.62 |100.0 |99.99 |636 |202.83 |
2013-08-02|109347.0 |100.0 |99.99 |558 |195.96 |
2013-08-03|95266.89 |100.0 |99.99 |485 |196.43 |
2013-08-04|90930.98 |100.0 |99.99 |452 |201.17 |
2013-08-05|75882.31 |100.0 |99.99 |392 |193.58 |
2013-08-06|120573.66 |100.0 |99.99 |626 |192.61 |
2013-08-07|103351.39 |100.0 |99.99 |532 |194.27 |
2013-08-08|76501.66 |100.0 |99.99 |401 |190.78 |
2013-08-09|62316.47 |100.0 |99.99 |322 |193.53 |
2013-08-10|129574.8 |100.0 |99.99 |639 |202.78 |
2013-08-11|71149.59 |100.0 |99.99 |347 |205.04 |
2013-08-12|121883.33 |100.0 |99.99 |613 |198.83 |
2013-08-13|39874.52 |100.0 |99.99 |204 |195.46 |
+----------+------------+---------+---------+--------------------+---------------+
only showing top 20 rows

###### Problem statement-2: Get sales data per month

In [12]:
SalesDataPerMonth = hiveContext.sql('''select substring(otl.order_date,0,7) as ordermonth,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales, 
                                   max(oitl.order_item_subtotal) as Max_sales, 
                                   count(oitl.order_item_subtotal) as Numberofsales_permonth, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_permonth 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                   group by ordermonth \
                                   order by ordermonth''')
SalesDataPerMonth.coalesce(1).write.csv("/FileStore/tables/SalesDataPerMonth", compression="none", header ='true')
SalesDataPerMonth.show(20,False)

+----------+------------+---------+---------+----------------------+-----------------+
ordermonth|Totalrevenue|Min_sales|Max_sales|Numberofsales_permonth|avgsales_permonth|
+----------+------------+---------+---------+----------------------+-----------------+
2013-07 |764782.19 |100.0 |99.99 |3821 |200.15 |
2013-08 |2828658.7 |100.0 |99.99 |14268 |198.25 |
2013-09 |2934527.27 |100.0 |999.99 |14694 |199.71 |
2013-10 |2624600.61 |100.0 |99.99 |13216 |198.59 |
2013-11 |3168656.03 |100.0 |99.99 |15905 |199.22 |
2013-12 |2932964.27 |100.0 |999.99 |14729 |199.13 |
2014-01 |2924447.01 |100.0 |999.99 |14666 |199.4 |
2014-02 |2778663.66 |100.0 |999.99 |14034 |198.0 |
2014-03 |2862492.21 |100.0 |99.99 |14350 |199.48 |
2014-04 |2807789.8 |100.0 |999.99 |14192 |197.84 |
2014-05 |2753078.22 |100.0 |999.99 |13754 |200.17 |
2014-06 |2703463.44 |100.0 |999.99 |13305 |203.19 |
2014-07 |2238496.52 |100.0 |999.99 |11264 |198.73 |
+----------+------------+---------+---------+----------------------+-----------------+

###### Problem statement-3: Get sales data per year

In [14]:
SalesDataPerYear = hiveContext.sql('''select substring(otl.order_date,0,4) as orderyear,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales, 
                                   max(oitl.order_item_subtotal) as Max_sales, 
                                   count(oitl.order_item_subtotal) as Numberofsales_year, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_peryear 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                   group by orderyear \
                                   order by orderyear''')
SalesDataPerYear.coalesce(1).write.csv("/FileStore/tables/SalesDataPerYear", compression="none", header ='true')
SalesDataPerYear.show(20,False)

+---------+-------------+---------+---------+------------------+----------------+
orderyear|Totalrevenue |Min_sales|Max_sales|Numberofsales_year|avgsales_peryear|
+---------+-------------+---------+---------+------------------+----------------+
2013 |1.525418907E7|100.0 |999.99 |76633 |199.06 |
2014 |1.906843086E7|100.0 |999.99 |95565 |199.53 |
+---------+-------------+---------+---------+------------------+----------------+

###### Problem statement-4:Get monthly sales data from third quarter of 2013 and first quarter of 2014 when Max sales >100

In [16]:
SalesDataPerMonthquarter = hiveContext.sql('''select substring(otl.order_date,0,7) as ordermonth,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales, 
                                   max(oitl.order_item_subtotal) as Max_sales, 
                                   count(oitl.order_item_subtotal) as Numberofsales_permonth, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_permonth 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                   where (CAST(otl.order_date AS DATE)>='2013-07-01' AND CAST(otl.order_date AS DATE)<='2013-09-30') 
                                   OR( CAST(otl.order_date AS DATE)>='2014-01-01'AND CAST(otl.order_date AS DATE)<='2014-03-31')
                                   group by ordermonth 
                                   having Max_sales > 100
                                   order by ordermonth''')
SalesDataPerMonthquarter.coalesce(1).write.csv("/FileStore/tables/SalesDataPerMonthquarter", compression="none", header ='true')
SalesDataPerMonthquarter.show(20,False)

+----------+------------+---------+---------+----------------------+-----------------+
ordermonth|Totalrevenue|Min_sales|Max_sales|Numberofsales_permonth|avgsales_permonth|
+----------+------------+---------+---------+----------------------+-----------------+
2013-09 |2934527.27 |100.0 |999.99 |14694 |199.71 |
2014-01 |2924447.01 |100.0 |999.99 |14666 |199.4 |
2014-02 |2778663.66 |100.0 |999.99 |14034 |198.0 |
+----------+------------+---------+---------+----------------------+-----------------+

###### Problem statement-5:Get monthly sales data from last quarter of 2013 where soldquantity_permonth>25000

In [18]:
SalesDataPerMonthquant = hiveContext.sql('''select substring(otl.order_date,0,7) as ordermonth,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales,  
                                   max(oitl.order_item_subtotal) as Max_sales,
                                   count(oitl.order_item_subtotal) as Numberofsales_permonth, 
                                   sum(oitl.order_item_quantity) as  soldquantity_permonth,
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_permonth 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   where CAST(otl.order_date AS DATE)>='2013-10-01' AND CAST(otl.order_date AS DATE)<='2013-12-31'
                                   group by ordermonth
                                   having soldquantity_permonth>25000
                                   order by ordermonth''')
SalesDataPerMonthquant.coalesce(1).write.csv("/FileStore/tables/SalesDataPerMonthquant", compression="none", header ='true')
SalesDataPerMonthquant.show(20,False)

+----------+------------+---------+---------+----------------------+---------------------+-----------------+
ordermonth|Totalrevenue|Min_sales|Max_sales|Numberofsales_permonth|soldquantity_permonth|avgsales_permonth|
+----------+------------+---------+---------+----------------------+---------------------+-----------------+
2013-10 |2624600.61 |100.0 |99.99 |13216 |28904.0 |198.59 |
2013-11 |3168656.03 |100.0 |99.99 |15905 |34705.0 |199.22 |
2013-12 |2932964.27 |100.0 |999.99 |14729 |31921.0 |199.13 |
+----------+------------+---------+---------+----------------------+---------------------+-----------------+

###### Problem statement-6:Get daily sales data from fourth quarter of 2013 and second quarter of 2014 where averagesale > 200 and Numberofsales >=600

In [20]:
SalesDataPerdayavgsales = hiveContext.sql('''select substring(otl.order_date,0,10) as orderdate,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales, 
                                   max(oitl.order_item_subtotal) as Max_sales, 
                                   count(oitl.order_item_subtotal) as Numberofsales, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                   where (CAST(otl.order_date AS DATE)>='2013-10-01' AND CAST(otl.order_date AS DATE)<='2013-12-31') 
                                   OR( CAST(otl.order_date AS DATE)>='2014-04-01' AND CAST(otl.order_date AS DATE)<='2014-06-30')
                                   group by orderdate 
                                   having avgsales > 200 and Numberofsales >=600
                                   order by orderdate''')
SalesDataPerdayavgsales.coalesce(1).write.csv("/FileStore/tables/SalesDataPerdayavgsales", compression="none", header ='true')
SalesDataPerdayavgsales.show(20,False)

+----------+------------+---------+---------+-------------+--------+
orderdate |Totalrevenue|Min_sales|Max_sales|Numberofsales|avgsales|
+----------+------------+---------+---------+-------------+--------+
2013-10-06|125221.77 |100.0 |99.99 |622 |201.32 |
2013-10-13|145561.99 |100.0 |99.99 |727 |200.22 |
2013-11-01|121736.89 |100.0 |99.99 |606 |200.89 |
2013-11-03|175599.2 |100.0 |99.99 |860 |204.19 |
2013-11-06|124044.93 |100.0 |99.99 |613 |202.36 |
2013-11-14|149271.96 |100.0 |99.99 |743 |200.9 |
2013-11-22|126926.43 |100.0 |99.99 |622 |204.06 |
2013-11-23|128024.84 |100.0 |99.99 |629 |203.54 |
2013-11-29|136296.63 |100.0 |99.99 |676 |201.62 |
2013-11-30|125511.03 |100.0 |99.99 |620 |202.44 |
2013-12-01|126193.28 |100.0 |99.99 |624 |202.23 |
2013-12-07|127089.81 |100.0 |99.99 |634 |200.46 |
2013-12-20|134371.41 |100.0 |99.99 |668 |201.15 |
2014-04-03|127594.93 |100.0 |99.99 |623 |204.81 |
2014-05-06|137077.97 |100.0 |99.99 |676 |202.78 |
2014-05-12|127265.59 |100.0 |99.99 |622 |204.61 |
2014-05-16|139834.47 |100.0 |99.99 |670 |208.71 |
2014-06-08|121925.54 |100.0 |99.99 |606 |201.2 |
2014-06-13|128392.34 |100.0 |99.99 |626 |205.1 |
2014-06-19|147183.83 |100.0 |99.99 |709 |207.59 |
+----------+------------+---------+---------+-------------+--------+
only showing top 20 rows

###### Problem statement-7:Get daily sales data  where soldquantity_perday > 1500

In [22]:
SalesDataPerdaysoldquant = hiveContext.sql('''select substring(otl.order_date,0,10) as orderdate,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales,  
                                   max(oitl.order_item_subtotal) as Max_sales,
                                   count(oitl.order_item_subtotal) as Numberofsales_perday, 
                                   sum(oitl.order_item_quantity) as  soldquantity_perday,
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_perday 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   group by orderdate
                                   having soldquantity_perday>1500
                                   order by orderdate''')
SalesDataPerdaysoldquant.coalesce(1).write.csv("/FileStore/tables/SalesDataPerdaysoldquant", compression="none", header ='true')
SalesDataPerdaysoldquant.show(25,False)

+----------+------------+---------+---------+--------------------+-------------------+---------------+
orderdate |Totalrevenue|Min_sales|Max_sales|Numberofsales_perday|soldquantity_perday|avgsales_perday|
+----------+------------+---------+---------+--------------------+-------------------+---------------+
2013-07-26|136520.17 |100.0 |99.99 |694 |1511.0 |196.71 |
2013-07-29|137287.09 |100.0 |99.99 |666 |1518.0 |206.14 |
2013-09-14|135308.52 |100.0 |99.99 |681 |1518.0 |198.69 |
2013-09-25|141775.64 |100.0 |99.99 |730 |1609.0 |194.21 |
2013-09-27|136537.86 |100.0 |99.99 |671 |1509.0 |203.48 |
2013-10-04|138079.62 |100.0 |99.99 |709 |1591.0 |194.75 |
2013-10-13|145561.99 |100.0 |99.99 |727 |1574.0 |200.22 |
2013-11-03|175599.2 |100.0 |99.99 |860 |1822.0 |204.19 |
2013-11-05|138528.34 |100.0 |99.99 |704 |1622.0 |196.77 |
2013-11-14|149271.96 |100.0 |99.99 |743 |1700.0 |200.9 |
2013-11-24|142234.02 |100.0 |99.99 |726 |1526.0 |195.91 |
2013-12-26|143012.23 |100.0 |999.99 |734 |1567.0 |194.84 |
2014-01-05|138550.94 |100.0 |99.99 |704 |1575.0 |196.81 |
2014-02-01|153561.8 |100.0 |99.99 |757 |1660.0 |202.86 |
2014-02-19|141857.25 |100.0 |99.99 |713 |1513.0 |198.96 |
2014-04-27|132838.34 |100.0 |99.99 |687 |1524.0 |193.36 |
2014-06-19|147183.83 |100.0 |99.99 |709 |1574.0 |207.59 |
2014-06-27|139952.39 |100.0 |99.99 |692 |1590.0 |202.24 |
2014-07-15|136902.59 |100.0 |99.99 |704 |1501.0 |194.46 |
+----------+------------+---------+---------+--------------------+-------------------+---------------+

###### Problem statement-8:Get final qurater of 2013 sales data of TOP5 revenue generating order for each month

In [24]:
SalesDataPermonthquarterly =  hiveContext.sql('''select * from(select substring(otl.order_date,0,7) as ordermonth,otl.order_id,round(sum(oitl.order_item_subtotal),2) as Totalrevenue,               
                                   rank() over(partition by substring(otl.order_date,0,7) order by round(sum(oitl.order_item_subtotal),2) desc) ranking_order
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                   where (CAST(otl.order_date AS DATE)>='2013-10-01' AND CAST(otl.order_date AS DATE)<='2013-12-31') 
                                   group by otl.order_id,otl.order_date 
                                   order by ordermonth )
                                   where ranking_order<=5 order by 1,3 desc''')
SalesDataPermonthquarterly.coalesce(1).write.csv("/FileStore/tables/SalesDataPermonthquarterly", compression="none", header ='true')
SalesDataPermonthquarterly.show(20,False)

+----------+--------+------------+-------------+
ordermonth|order_id|Totalrevenue|ranking_order|
+----------+--------+------------+-------------+
2013-10 |68736 |2259.95 |1 |
2013-10 |14539 |1899.9 |2 |
2013-10 |59652 |1799.81 |3 |
2013-10 |13445 |1699.88 |4 |
2013-10 |14775 |1649.8 |5 |
2013-11 |18735 |1819.86 |1 |
2013-11 |17185 |1699.91 |2 |
2013-11 |19754 |1699.81 |3 |
2013-11 |60794 |1649.84 |4 |
2013-11 |17659 |1649.82 |5 |
2013-12 |68766 |2699.9 |1 |
2013-12 |22030 |2039.8 |2 |
2013-12 |23631 |1829.86 |3 |
2013-12 |61289 |1799.89 |4 |
2013-12 |23425 |1759.9 |5 |
+----------+--------+------------+-------------+

###### Problem statement-9:Get average revenue per day and all the orders which are more than average.

In [26]:
SalesDataPerdayavgsales =  hiveContext.sql('''SELECT order_date, order_id, round(order_revenue,2), round(avg_revenue,2) FROM 
                                          (SELECT otl.order_date, otl.order_id, sum(oitl.order_item_subtotal) order_revenue, 
                                           AVG( sum(oitl.order_item_subtotal)) OVER (PARTITION BY otl.order_date ) avg_revenue  
                                           from orders otl join order_items oitl WHERE otl.order_id = oitl.order_item_order_id GROUP BY  otl.order_date, otl.order_id) 
                                           WHERE order_revenue >= avg_revenue ORDER BY 1, 3 DESC''')
SalesDataPerdayavgsales.coalesce(1).write.csv("/FileStore/tables/SalesDataPerdayavgsales", compression="none", header ='true')
SalesDataPerdayavgsales.show(20,False)

+-----------------------+--------+-----------------------+---------------------+
order_date |order_id|round(order_revenue, 2)|round(avg_revenue, 2)|
+-----------------------+--------+-----------------------+---------------------+
2013-07-25 00:00:00.000|57779 |1649.8 |587.53 |
2013-07-25 00:00:00.000|84 |1499.87 |587.53 |
2013-07-25 00:00:00.000|12 |1299.87 |587.53 |
2013-07-25 00:00:00.000|73 |1279.65 |587.53 |
2013-07-25 00:00:00.000|28 |1159.9 |587.53 |
2013-07-25 00:00:00.000|62 |1149.94 |587.53 |
2013-07-25 00:00:00.000|57764 |1149.92 |587.53 |
2013-07-25 00:00:00.000|5 |1129.86 |587.53 |
2013-07-25 00:00:00.000|57788 |1119.86 |587.53 |
2013-07-25 00:00:00.000|29 |1109.85 |587.53 |
2013-07-25 00:00:00.000|99 |1099.93 |587.53 |
2013-07-25 00:00:00.000|57757 |1099.87 |587.53 |
2013-07-25 00:00:00.000|57782 |1049.85 |587.53 |
2013-07-25 00:00:00.000|57763 |1029.9 |587.53 |
2013-07-25 00:00:00.000|57765 |966.9 |587.53 |
2013-07-25 00:00:00.000|15 |925.91 |587.53 |
2013-07-25 00:00:00.000|11 |919.79 |587.53 |
2013-07-25 00:00:00.000|57759 |909.97 |587.53 |
2013-07-25 00:00:00.000|101 |899.94 |587.53 |
2013-07-25 00:00:00.000|63 |899.92 |587.53 |
+-----------------------+--------+-----------------------+---------------------+
only showing top 20 rows

###### Problem statement-10:Percentage of order items in Order revenue

In [28]:
SalesDataPcntorders =  hiveContext.sql('''SELECT order_item_order_id, order_item_id, order_item_subtotal, 
                                            round(SUM(order_item_subtotal) OVER(PARTITION BY order_item_order_id),2) order_revenue , 
                                           (round(order_item_subtotal /SUM(order_item_subtotal) OVER(PARTITION BY order_item_order_id),2)) percentage FROM order_items ORDER BY 1''')
SalesDataPcntorders.coalesce(1).write.csv("/FileStore/tables/SalesDataPcntorders", compression="none", header ='true')
SalesDataPcntorders.show(20,False)

+-------------------+-------------+-------------------+-------------+----------+
order_item_order_id|order_item_id|order_item_subtotal|order_revenue|percentage|
+-------------------+-------------+-------------------+-------------+----------+
1 |1 |299.98 |299.98 |1.0 |
10 |24 |199.99 |651.92 |0.31 |
10 |25 |99.96 |651.92 |0.15 |
10 |28 |199.99 |651.92 |0.31 |
10 |27 |21.99 |651.92 |0.03 |
10 |26 |129.99 |651.92 |0.2 |
100 |226 |129.99 |549.94 |0.24 |
100 |228 |299.97 |549.94 |0.55 |
100 |227 |119.98 |549.94 |0.22 |
1000 |2494 |99.96 |279.93 |0.36 |
1000 |2493 |49.98 |279.93 |0.18 |
1000 |2495 |129.99 |279.93 |0.46 |
10000 |24976 |129.99 |209.97 |0.62 |
10000 |24975 |79.98 |209.97 |0.38 |
10001 |24978 |129.99 |779.89 |0.17 |
10001 |24977 |29.97 |779.89 |0.04 |
10001 |24980 |119.98 |779.89 |0.15 |
10001 |24979 |499.95 |779.89 |0.64 |
10002 |24981 |129.99 |129.99 |1.0 |
10003 |24982 |99.96 |99.96 |1.0 |
+-------------------+-------------+-------------------+-------------+----------+
only showing top 20 rows